## НАШ ПРОЕКТИК

# 1. Получение данных


In [15]:
import requests
import pandas as pd
from datetime import datetime

# Геокоординаты для нужных городов
city_coords = {
    'Moscow': (55.7558, 37.6176),
    'Saint_Petersburg': (59.9343, 30.3351),
    'Nizhny_Novgorod': (56.2965, 43.9361),
    'Kazan': (55.7963, 49.1088),
    'Ufa': (54.7388, 55.9721),
    'Novosibirsk': (55.0084, 82.9357),
    'Vladivostok': (43.1198, 131.8869)
}

# Функция получения данных из Open-Meteo
def get_weather_open_meteo(city: str, lat: float, lon: float,
                           start_date='2023-01-01', end_date='2023-12-31',
                           hourly=True) -> pd.DataFrame:
    base_url = 'https://archive-api.open-meteo.com/v1/archive'
    params = {
        'latitude': lat,
        'longitude': lon,
        'start_date': start_date,
        'end_date': end_date,
        'timezone': 'auto'
    }

    if hourly:
        params['hourly'] = 'temperature_2m'
    else:
        params['daily'] = 'temperature_2m_mean'

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception(f'Ошибка {response.status_code}: {response.text}')
    
    data = response.json()
    
    if hourly:
        time_list = data['hourly']['time']
        temp_list = data['hourly']['temperature_2m']
    else:
        time_list = data['daily']['time']
        temp_list = data['daily']['temperature_2m_mean']
    
    df = pd.DataFrame({
        'datetime': pd.to_datetime(time_list),
        'temperature': temp_list
    })
    df['city'] = city
    return df

# Сбор всех данных
hourly_data = []
daily_data = []

for city, (lat, lon) in city_coords.items():
    print(f"[⏱] Загружаем почасовые данные для: {city}")
    df_hourly = get_weather_open_meteo(city, lat, lon, hourly=True)
    hourly_data.append(df_hourly)
    
    print(f"[📆] Загружаем ежедневные данные для: {city}")
    df_daily = get_weather_open_meteo(city, lat, lon, hourly=False)
    daily_data.append(df_daily)

# Объединяем все города
df_hourly_all = pd.concat(hourly_data).reset_index(drop=True)
df_daily_all = pd.concat(daily_data).reset_index(drop=True)

# Агрегация по месяцам
df_monthly_all = (
    df_daily_all.copy()
    .assign(month=lambda df: df['datetime'].dt.to_period('M'))
    .groupby(['city', 'month'])
    .agg(temperature=('temperature', 'mean'))
    .reset_index()
)
df_monthly_all['month'] = df_monthly_all['month'].astype(str)

# Сохраняем все 3 таблицы
df_hourly_all.to_csv("weather_2023_hourly.csv", index=False)
df_daily_all.to_csv("weather_2023_daily.csv", index=False)
df_monthly_all.to_csv("weather_2023_monthly.csv", index=False)

# Функция для присвоения сезона по месяцу
def get_season(month: int) -> str:
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

# Почасовые данные
df_hourly_all['month'] = df_hourly_all['datetime'].dt.month
df_hourly_all['season'] = df_hourly_all['month'].apply(get_season)

# Ежедневные данные
df_daily_all['month'] = df_daily_all['datetime'].dt.month
df_daily_all['season'] = df_daily_all['month'].apply(get_season)

# Ежемесячные данные
df_monthly_all['month_num'] = pd.to_datetime(df_monthly_all['month']).dt.month
df_monthly_all['season'] = df_monthly_all['month_num'].apply(get_season)

# Проверка
print("\n📌 Почасовые данные с сезонами:")
print(df_hourly_all[['datetime', 'temperature', 'city', 'season']].head())

print("\n📌 Ежедневные данные с сезонами:")
print(df_daily_all[['datetime', 'temperature', 'city', 'season']].head())

print("\n📌 Ежемесячные данные с сезонами:")
print(df_monthly_all[['month', 'temperature', 'city', 'season']].head())

[⏱] Загружаем почасовые данные для: Moscow
[📆] Загружаем ежедневные данные для: Moscow
[⏱] Загружаем почасовые данные для: Saint_Petersburg
[📆] Загружаем ежедневные данные для: Saint_Petersburg
[⏱] Загружаем почасовые данные для: Nizhny_Novgorod
[📆] Загружаем ежедневные данные для: Nizhny_Novgorod
[⏱] Загружаем почасовые данные для: Kazan
[📆] Загружаем ежедневные данные для: Kazan
[⏱] Загружаем почасовые данные для: Ufa
[📆] Загружаем ежедневные данные для: Ufa
[⏱] Загружаем почасовые данные для: Novosibirsk
[📆] Загружаем ежедневные данные для: Novosibirsk
[⏱] Загружаем почасовые данные для: Vladivostok
[📆] Загружаем ежедневные данные для: Vladivostok

📌 Почасовые данные с сезонами:
             datetime  temperature    city  season
0 2023-01-01 00:00:00         -0.4  Moscow  winter
1 2023-01-01 01:00:00         -0.2  Moscow  winter
2 2023-01-01 02:00:00          0.0  Moscow  winter
3 2023-01-01 03:00:00          0.3  Moscow  winter
4 2023-01-01 04:00:00          0.5  Moscow  winter

📌 

# 2. Предобработка данных 

# 3. Анализ корреляции

3.1 Рассчет корреляций

3.2 Анализ корреляций 

# 4. Построение сетевых моделей 

4.1. Создание графов

4.2. Выделение клик и независимых множеств

# 5. Сравнение результатов